# IMDB (Dataset)

In [ ]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  3716k      0  0:00:22  0:00:22 --:--:-- 3920k


In [ ]:
!rm -r aclImdb/train/unsup

In [ ]:
from tensorflow import keras
batch_size = 32

In [ ]:
train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    seed = 2023,
    subset="training",
)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


In [ ]:
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    seed = 2023,
    subset="validation",
)

Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [ ]:
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test",
    batch_size=batch_size
)

Found 25000 files belonging to 2 classes.


In [ ]:
# Visualizar características del primer lote (batch)
for inputs, targets in train_ds:
  print("inputs.shape:", inputs.shape)
  print("inputs.dtype:", inputs.dtype)
  print("targets.shape:", targets.shape)
  print("targets.dtype:", targets.dtype)
  print("inputs[0]:", inputs[0])
  print("targets[0]:", targets[0])
  break

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b"This movie took my breath away at some points, I simply loved it! <br /><br />I admit that the character dialogs and storyline could have been done a bit better, but hey, this is just a simple (short) story of a couple of guys trying to slain a dragon, there's nothing more to it!<br /><br />The overall design, atmosphere, the beautiful landscapes... it's all just magical! <br /><br />They've put a lot of love in this movie. Character designs were great and funny. A bit Tim Burton-ish if you like. I can recommend this movie to anyone interested in great design, displayed in a simple small, but lovely story.", shape=(), dtype=string)
targets[0]: tf.Tensor(1, shape=(), dtype=int32)


# TextVectorization

Pre-procesamiento


*   Limitar a las 20000 palabras más frecuentes
*   Multi-hot enconding
*   Texto plano sin etiquetas
*   Indexar el dataset de acuerdo con el vocabulario

https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

text_vectorization = TextVectorization(
    ngrams=1,
    max_tokens=20000,
    output_mode="multi_hot",
)

# Extraer solo el texto (features) de los datos de entrada para calcular el vocabulario
text_only_train_ds = train_ds.map(lambda x, y: x)

# Calcular el vocabulario de los tokens de entrada
text_vectorization.adapt(text_only_train_ds)

# Definir los datasets
# Procesar los features de entrada con text_vectorization y conservar la etiqueta
Unigram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

Unigram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

Unigram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

Visualizar las características de la salida después del TextVectorization:


*   Vectores de 20000 elementos
*   Valores (elementos) del vector tipo Int32 (unos y ceros)

In [ ]:
for inputs, targets in Unigram_train_ds:
  print("inputs.shape:", inputs.shape)
  print("inputs.dtype:", inputs.dtype)
  print("targets.shape:", targets.shape)
  print("targets.dtype:", targets.dtype)
  print("inputs[0]:", inputs[0])
  print("targets[0]:", targets[0])
  break

inputs.shape: (32, 20000)
inputs.dtype: <dtype: 'int64'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor([1 1 1 ... 0 0 0], shape=(20000,), dtype=int64)
targets[0]: tf.Tensor(1, shape=(), dtype=int32)


# Modelo

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Input, Dropout

max_tokens=20000
No_unidades=16

model_MLP = Sequential([
    Input(shape=(max_tokens,)),
    Dense(No_unidades, activation="relu"),
    Dropout(0.5),
    Dense(1, activation="sigmoid")
])

model_MLP.compile(
    optimizer='rmsprop',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model_MLP.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 16)                  │         320,016 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 320,033 (1.22 MB)

 Trainable params: 320,033 (1.22 MB)

 Non-trainable params: 0 (0.00 B)

# Entrenamiento

In [ ]:
# Utilización de caché
model_MLP.fit(Unigram_train_ds.cache(),
          validation_data=Unigram_val_ds.cache(),
          epochs=10,
          #callbacks=callbacks
              )

Epoch 1/10
625/625 [==============================] - 10s 16ms/step - loss: 0.4201 - accuracy: 0.8217 - val_loss: 0.2904 - val_accuracy: 0.8856
Epoch 2/10
625/625 [==============================] - 4s 7ms/step - loss: 0.2846 - accuracy: 0.8941 - val_loss: 0.2722 - val_accuracy: 0.8916
Epoch 3/10
625/625 [==============================] - 5s 9ms/step - loss: 0.2566 - accuracy: 0.9103 - val_loss: 0.2788 - val_accuracy: 0.8926
Epoch 4/10
625/625 [==============================] - 4s 7ms/step - loss: 0.2392 - accuracy: 0.9166 - val_loss: 0.2875 - val_accuracy: 0.8908
Epoch 5/10
625/625 [==============================] - 5s 8ms/step - loss: 0.2233 - accuracy: 0.9258 - val_loss: 0.2984 - val_accuracy: 0.8912
Epoch 6/10
625/625 [==============================] - 4s 6ms/step - loss: 0.2248 - accuracy: 0.9280 - val_loss: 0.3092 - val_accuracy: 0.8876
Epoch 7/10
625/625 [==============================] - 5s 8ms/step - loss: 0.2149 - accuracy: 0.9290 - val_loss: 0.3188 - val_accuracy: 0.8902
Epoc

#Evaluación del modelo

Sobre datos test

In [ ]:
#model_MLP = keras.models.load_model("binary_1gram.keras")

print(f"Test acc: {model_MLP.evaluate(Unigram_test_ds)[1]:.3f}")

782/782 [==============================] - 6s 8ms/step - loss: 0.3750 - accuracy: 0.8742
Test acc: 0.874


# Predicción

In [ ]:
# Para la predicción, es necesario incoprpar al modelo la capa de pre-procesamiento

inputs = keras.Input(shape=(1,), dtype="string")
processed_inputs = text_vectorization(inputs)
outputs = model_MLP(processed_inputs)
inference_model = keras.Model(inputs, outputs)

In [ ]:
raw_text_data = tf.convert_to_tensor([
    ["That was an excellent movie, I loved it."],
    ])
predictions = inference_model(raw_text_data)
print(f"{float(predictions[0] * 100):.2f} percent positive")

90.83 percent positive


# Bigram

In [ ]:
text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="multi_hot",
)

# Calcular el vocabulario de los tokens de entrada
text_vectorization.adapt(text_only_train_ds)

# Definir los datasets
# Procesar los features de entrada con text_vectorization y conservar la etiqueta
Bigram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

Bigram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

Bigram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

In [ ]:
model_MLP2 = Sequential([
    Input(shape=(max_tokens,)),
    Dense(No_unidades, activation="relu"),
    Dropout(0.5),
    Dense(1, activation="sigmoid")
])

model_MLP2.compile(
    optimizer='rmsprop',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Utilización de caché
model_MLP2.fit(Bigram_train_ds.cache(),
          validation_data=Bigram_val_ds.cache(),
          epochs=10,
          #callbacks=callbacks
              )

Epoch 1/10
625/625 [==============================] - 11s 16ms/step - loss: 0.4045 - accuracy: 0.8299 - val_loss: 0.2711 - val_accuracy: 0.8976
Epoch 2/10
625/625 [==============================] - 4s 6ms/step - loss: 0.2653 - accuracy: 0.9061 - val_loss: 0.2616 - val_accuracy: 0.9008
Epoch 3/10
625/625 [==============================] - 5s 8ms/step - loss: 0.2265 - accuracy: 0.9247 - val_loss: 0.2723 - val_accuracy: 0.9012
Epoch 4/10
625/625 [==============================] - 4s 6ms/step - loss: 0.2090 - accuracy: 0.9348 - val_loss: 0.3011 - val_accuracy: 0.8994
Epoch 5/10
625/625 [==============================] - 5s 8ms/step - loss: 0.1907 - accuracy: 0.9416 - val_loss: 0.3042 - val_accuracy: 0.8990
Epoch 6/10
625/625 [==============================] - 4s 7ms/step - loss: 0.1850 - accuracy: 0.9459 - val_loss: 0.3480 - val_accuracy: 0.8966
Epoch 7/10
625/625 [==============================] - 5s 8ms/step - loss: 0.1753 - accuracy: 0.9492 - val_loss: 0.3586 - val_accuracy: 0.8948
Epoc

In [ ]:
print(f"Test acc: {model_MLP2.evaluate(Bigram_test_ds)[1]:.3f}")

782/782 [==============================] - 7s 8ms/step - loss: 0.3783 - accuracy: 0.8845
Test acc: 0.884


# Bigram + TF_IDF

TF_IDF: taking the histogram of the words over the text

(Term frequency, inverse document frequency): how often the term comes up across the dataset

In [ ]:
text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="tf_idf",
)

# Extraer solo el texto (features) de los datos de entrada
text_only_train_ds = train_ds.map(lambda x, y: x)

# Calcular el vocabulario de los tokens de entrada
text_vectorization.adapt(text_only_train_ds)

# Definir los datasets
# Procesar los features de entrada con text_vectorization y conservar la etiqueta
TFBigram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

TFBigram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

TFBigram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

In [ ]:
model_MLP3 = Sequential([
    Input(shape=(max_tokens,)),
    Dense(No_unidades, activation="relu"),
    Dropout(0.5),
    Dense(1, activation="sigmoid")
])

model_MLP3.compile(
    optimizer='rmsprop',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Utilización de caché
model_MLP3.fit(TFBigram_train_ds.cache(),
          validation_data=TFBigram_val_ds.cache(),
          epochs=10,
          #callbacks=callbacks
              )

Epoch 1/10
625/625 [==============================] - 11s 16ms/step - loss: 0.4456 - accuracy: 0.8133 - val_loss: 0.3665 - val_accuracy: 0.8524
Epoch 2/10
625/625 [==============================] - 4s 7ms/step - loss: 0.2936 - accuracy: 0.8924 - val_loss: 0.2786 - val_accuracy: 0.8990
Epoch 3/10
625/625 [==============================] - 5s 8ms/step - loss: 0.2515 - accuracy: 0.9050 - val_loss: 0.3090 - val_accuracy: 0.8858
Epoch 4/10
625/625 [==============================] - 5s 8ms/step - loss: 0.2343 - accuracy: 0.9129 - val_loss: 0.3292 - val_accuracy: 0.8886
Epoch 5/10
625/625 [==============================] - 5s 7ms/step - loss: 0.2199 - accuracy: 0.9158 - val_loss: 0.2973 - val_accuracy: 0.8910
Epoch 6/10
625/625 [==============================] - 6s 9ms/step - loss: 0.2043 - accuracy: 0.9220 - val_loss: 0.3162 - val_accuracy: 0.8962
Epoch 7/10
625/625 [==============================] - 4s 7ms/step - loss: 0.1995 - accuracy: 0.9211 - val_loss: 0.3829 - val_accuracy: 0.8858
Epoc

In [ ]:
print(f"Test acc: {model_MLP3.evaluate(TFBigram_test_ds)[1]:.3f}")

782/782 [==============================] - 6s 7ms/step - loss: 0.3606 - accuracy: 0.8760
Test acc: 0.876


# Exportar csv to txt files

En excel separar archivos xlsx por clase y luego guardar en cada carpeta

In [ ]:
import pandas as pd
df = pd.read_csv('my_file.csv')

for i in range(df.shape[0]):
  content = df['Name'][i:i+1].to_string(index=False)
  print(content, file=open('my_file' + str(i) + '.txt', 'w'), end="")